In [21]:
import pandas as pd
import numpy as np 
import random

In [22]:
data = pd.read_csv('Live.csv')
columna1 = 'num_reactions'
columna2 ='num_comments'
data = data[data[columna1] != 0]
data = data[data[columna2] != 0]
columnas_seleccionadas = data[[columna1, columna2]]
print(columnas_seleccionadas)

      num_reactions  num_comments
0               529           512
2               227           236
5               217             6
6               503           614
7               295           453
...             ...           ...
7030             91             2
7032             10             1
7040             93            26
7043            196             2
7048            351            12

[4930 rows x 2 columns]


In [24]:
def euclidean_distance(point1, point2):
    sum_squared_distance = sum((x - y) ** 2 for x, y in zip(point1, point2))
    return sum_squared_distance ** 0.5

def initialize_centroids(data, k):
    centroids = random.sample(data, k)
    return centroids

def assign_to_clusters(data, centroids):
    clusters = [[] for _ in range(len(centroids))]
    
    for point in data:
        distances = [euclidean_distance(point, centroid) for centroid in centroids]
        cluster_index = distances.index(min(distances))
        clusters[cluster_index].append(point)
    
    return clusters

def update_centroids(clusters):
    centroids = []
    for cluster in clusters:
        cluster_center = [sum(point[i] for point in cluster) / len(cluster) for i in range(len(cluster[0]))]
        centroids.append(cluster_center)
    return centroids

def has_converged(old_centroids, new_centroids, tol=1e-4):
    return all(euclidean_distance(old, new) < tol for old, new in zip(old_centroids, new_centroids))

def k_means(data, k):
    centroids = initialize_centroids(data, k)
    converged = False
    
    while not converged:
        clusters = assign_to_clusters(data, centroids)
        new_centroids = update_centroids(clusters)
        converged = has_converged(centroids, new_centroids)
        centroids = new_centroids
    
    return centroids, clusters


data_list = columnas_seleccionadas.values.tolist()
k=3
centroids, clusters = k_means(data_list, k)

for i, centroid in enumerate(centroids):
    print(f"Centroide {i + 1}: {centroid}")
    print(f"Puntos en el cluster {i + 1}: {clusters[i]}")


Centroide 1: [161.54547542489664, 163.05006890215893]
Puntos en el cluster 1: [[529, 512], [227, 236], [217, 6], [503, 614], [295, 453], [203, 1], [170, 9], [210, 2], [222, 4], [313, 4], [209, 4], [346, 11], [332, 100], [135, 256], [150, 173], [221, 166], [152, 2], [234, 15], [227, 7], [75, 36], [90, 78], [145, 9], [253, 300], [45, 14], [125, 1], [119, 4], [293, 8], [128, 137], [84, 15], [66, 11], [171, 292], [90, 72], [139, 39], [236, 181], [225, 210], [275, 12], [372, 456], [445, 9], [128, 60], [186, 235], [309, 444], [77, 11], [119, 72], [100, 21], [240, 255], [164, 136], [202, 327], [211, 284], [148, 114], [97, 81], [279, 292], [263, 8], [177, 193], [517, 32], [289, 258], [66, 13], [112, 6], [206, 259], [362, 468], [184, 130], [216, 251], [272, 258], [248, 247], [132, 89], [226, 341], [35, 1], [219, 316], [94, 1], [259, 368], [62, 1], [127, 76], [149, 63], [134, 171], [392, 461], [310, 540], [14, 3], [293, 6], [236, 12], [312, 10], [24, 12], [16, 9], [33, 12], [675, 10], [737, 18],